In [1]:
import sys
from elasticsearch import Elasticsearch
import json
import time
import sys
# from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from bs4 import BeautifulSoup
import csv
import re
import numpy as np
from tqdm import tqdm
import pandas as pd

In [4]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def clean_text(sentance):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    return sentance

def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model
def get_glove_vectors_from_sentence(sentence, glove_model):
    cleaned_text=clean_text(sentence)
    sent_vec = np.zeros(300)
    sent=cleaned_text.split()
    sent_vec = np.array([0]*300)
    cnt_words=0
    for word in sent:
        
        try:
            vec = glove_model[word]
            sent_vec = sent_vec + vec
            cnt_words += 1
        except:
            pass
    if len(sent) != 0:
        sent_vec /= cnt_words
    return sent_vec
glove_model=load_glove_model("/Users/amansawarn/Documents/tech/iitd/Building-Search-and-Recommendations-Together/glove_6B/glove.6B.300d.txt")


Loading Glove Model
400000 words loaded!


In [27]:
# Search by Vec Similarity

def connect2ES():
    # connect to ES on localhost on port 9200
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
            print('Connected to ES!')
    else:
            print('Could not connect!')
            sys.exit()

    print("*********************************************************************************");
    return es

def keywordSearch(es, q):
    #Search by Keywords
    b={
            'query':{
                'match':{
                    "title":q
                }
            }
        }

    res= es.search(index='titles_questions_vectors-index',body=b)
    print("Keyword Search:\n")
    for hit in res['hits']['hits']:
        print("-"*40)
        print(str(hit['_score']) + "\t" + hit['_source']['title'] )

    print("*********************************************************************************");

    return


def sentenceSimilaritybyNN_300d(embeddings, es, sent):
    """
    {"title":title,
    "title_question":title_body, 
    "titlequestion__vector":title_body_vec }
    """
#     query_vector = tf.make_ndarray(tf.make_tensor_proto(embed([sent]))).tolist()[0]
    b = {"query" : {
                "script_score" : {
                    "query" : {
                        "match_all": {}
                    },
                    "script" : {
                        "source": "cosineSimilarity(params.query_vector, 'title_question_vector') + 1.0",
                        "params": {"query_vector": embeddings}
                    }
                }
             }
        }


    #print(json.dumps(b,indent=4))
    res= es.search(index='titles_questions_vectors-index',body=b)

    print("Semantic Similarity Search:\n")
    for hit in res['hits']['hits']:
        print("-"*40)
        print(str(hit['_score']) + "\t" + hit['_source']['title'] )

    print("*********************************************************************************");


In [28]:
es = connect2ES()

Connected to ES!
*********************************************************************************


In [29]:
query="how to install pip?"
embeddings=list(get_glove_vectors_from_sentence(query, glove_model))
# embeddings=[-0.03462885320186615, 0.00767511036247015, 0.004583781119436026, 0.028401656076312065, 0.01258331723511219, 0.037116874009370804, 0.013477616012096405, 0.022381523624062538, 0.03979786857962608, 0.024851974099874496, 0.036737024784088135, 0.03597935289144516, -0.0381510891020298, 0.08130659908056259, -0.05906254053115845, 0.08973553776741028, -0.03244425728917122, 0.051288917660713196, 0.02930707484483719, 0.022335924208164215, 0.020585650578141212, 0.04718436673283577, -0.05237151309847832, -0.043897032737731934, 0.015714094042778015, 0.02331877127289772, 0.03893255814909935, -0.05679837986826897, 0.03729335218667984, -0.049225278198719025, 0.06690701842308044, 0.07899188995361328, 0.016306092962622643, -0.02428351156413555, 0.019640468060970306, 0.03080165386199951, 0.07699830085039139, -0.014750823378562927, -0.051929473876953125, 0.0391777828335762, 0.034724123775959015, -0.03729329630732536, 0.0199520792812109, -0.019983848556876183, 0.06893950700759888, -0.01282433234155178, -0.018197476863861084, -0.0725010335445404, 0.08238957822322845, 0.038134943693876266, 0.04725194349884987, 0.08184657990932465, -0.038238804787397385, -0.040010299533605576, -0.05526154115796089, -0.04314691573381424, -0.017179878428578377, -0.04188328608870506, 0.06424654275178909, 0.08285296708345413, 0.013338451273739338, 0.02647676318883896, 0.0059562912210822105, -0.027724098414182663, 0.03596652299165726, -0.0713033601641655, -0.040550850331783295, -0.054018519818782806, -0.05968989431858063, 0.05485939979553223, 0.0054743546061217785, 0.04592319577932358, 0.04088946059346199, 0.009760835207998753, -0.024534588679671288, 0.03911426663398743, 0.034169845283031464, -0.009666499681770802, 0.04447918012738228, 0.05167559161782265, -0.025161610916256905, 0.05686209723353386, -0.024312159046530724, 0.0038878333289176226, -0.06046237796545029, 0.00990272406488657, 0.0021283847745507956, 0.035826608538627625, 0.07240117341279984, -0.03567688912153244, -0.07299879193305969, 0.0098195755854249, -0.07013959437608719, 0.05116822198033333, -0.07229223847389221, -0.008418512530624866, 0.034077126532793045, 0.06645393371582031, 0.027137843891978264, -0.08534674346446991, 0.03342875465750694, 0.0010534467874094844, -0.07039080560207367, -0.061267413198947906, 0.00017997882969211787, 0.0448671318590641, 0.002362641738727689, 0.030875999480485916, 0.05316605791449547, -0.04655775800347328, 0.055160440504550934, -0.014520411379635334, -0.03851637616753578, -0.023013180121779442, -0.043779704719781876, 0.02196790650486946, 0.019681503996253014, 0.025772398337721825, -0.07112425565719604, -0.041777562350034714, -0.08362290263175964, 0.011219008825719357, 0.041559576988220215, 0.042125992476940155, -0.028749441727995872, -0.026517637073993683, -0.015619627200067043, 0.0024774977937340736, 0.04005668684840202, 0.06414245814085007, -0.0193956159055233, 0.05626853182911873, -0.006098745856434107, -0.027311531826853752, -0.0020660171285271645, -0.055144451558589935, -0.0790662094950676, 0.013367146253585815, 0.013629199005663395, 0.032548774033784866, -0.052126847207546234, -0.07231476902961731, 0.045602746307849884, -0.022876564413309097, -0.01824858784675598, -0.07567820698022842, -0.05951853096485138, -0.0033317203633487225, -0.019438941031694412, -0.07292303442955017, 0.039319004863500595, -0.030262252315878868, 0.023188702762126923, 0.049401767551898956, -0.03795567527413368, 0.00596885709092021, 0.07696520537137985, -0.06870242953300476, -0.023071812465786934, 0.032319311052560806, 0.027019716799259186, 0.01199339423328638, -0.054047685116529465, -0.034860048443078995, 0.004286427516490221, 0.031746670603752136, -0.03668864071369171, 0.05324863642454147, -0.014945111237466335, 0.012569617480039597, -0.014295347034931183, -0.05456634238362312, -0.02866458334028721, -0.031186982989311218, 0.011176125146448612, -0.04290825501084328, -0.02617817372083664, -0.014771874062716961, -0.02310779318213463, 0.03021686151623726, 0.02180018089711666, -0.06680597364902496, -0.013131853193044662, 0.07586025446653366, 0.0587407648563385, 0.021105261519551277, -0.043579600751399994, -0.05390194058418274, -0.006434908602386713, 0.011121878400444984, 0.043450430035591125, -0.046666622161865234, 0.03760552406311035, 0.057415563613176346, -0.02668597176671028, 0.0025108535774052143, 0.07808926701545715, 0.09018489718437195, 0.03663500025868416, -0.049429137259721756, -0.05500154197216034, 0.0045248595997691154, -0.0663011223077774, 0.0767505019903183, -0.02247614786028862, -0.04814096540212631, 0.03155306726694107, 0.048325467854738235, -0.0317516028881073, 0.07029253989458084, -0.014762896113097668, 0.039758119732141495, -0.027383100241422653, 0.01077291090041399, -0.018674813210964203, 0.06772593408823013, -0.019307976588606834, 0.06167430803179741, -0.04702961817383766, 0.015139950439333916, 0.0035179078113287687, 0.03386974334716797, -0.027051519602537155, 0.03885263204574585, 0.03370879217982292, 0.030938170850276947, 0.03143477067351341, 0.033542290329933167, -0.06624799966812134, -0.04648910462856293, -0.04922991245985031, 0.03850280120968819, 0.03083956614136696, 0.0036689413245767355, -0.06498728692531586, 0.011056188493967056, -0.014391270466148853, -0.007774649187922478, -0.00781356543302536, 0.04289641231298447, 0.04494582489132881, -0.04518967121839523, 0.060684725642204285, -0.023170430213212967, 0.06990323215723038, 0.0877854973077774, 0.0027513860259205103, -0.05976095050573349, 0.09043174982070923, 0.013956253416836262, -0.09000561386346817, -0.015043241903185844, -0.03513569012284279, 0.01768846996128559, -0.04067947715520859, -0.014423196204006672, -0.030041474848985672, -0.06351054459810257, 0.02664155326783657, 0.03906116634607315, 0.0147284846752882, 0.01246387418359518, -0.09055300801992416, 0.07144160568714142, -0.05196592956781387, -0.020227991044521332, 0.0341203548014164, -0.08681684732437134, 0.02282768115401268, -0.004371015354990959, 0.011171799153089523, -0.017510192468762398, 0.08090528100728989, -0.041865479201078415, -0.01456389483064413, -0.020004156976938248, -0.009722404181957245, 0.03202565386891365, 0.0694165900349617, -0.02828393317759037, -0.032582838088274, -0.011106813326478004, -0.030484862625598907, 0.009067831560969353, -0.020938389003276825, -0.0230411347001791, -0.07012031972408295, 0.0541088804602623, -0.042449284344911575, 0.018649384379386902, 0.03883485123515129, -0.048480186611413956, 0.0344393365085125, -0.06893027573823929, 0.08569465577602386, -0.06986629217863083, 0.008478878997266293, 0.06473923474550247, -0.07792525738477707, 0.001722044195048511, 0.01306208036839962, 0.04006865620613098, -0.06954129040241241, -0.021065140143036842, -0.005601787474006414, 0.0076944660395383835, 0.032397426664829254, 0.057154640555381775, -0.05126655101776123, -0.012916426174342632, 0.03863914683461189, 0.04436832293868065, 0.009613048285245895, -0.03996099904179573, -0.0003503428597468883, 0.031744007021188736, -0.00216520088724792, 0.017528623342514038, -0.06498982012271881, 0.07106641680002213, -0.04223049059510231, 0.03721412643790245, -0.0073788901790976524, 0.06726673245429993, -0.03244607150554657, 0.05853502079844475, 0.005853424314409494, 0.018217280507087708, -0.0048925606533885, 0.07971342653036118, -0.04592495411634445, 0.05102983117103577, 0.07912196218967438, 0.07316495478153229, -0.017797192558646202, 0.048888444900512695, -0.06103556603193283, -0.03492056950926781, 0.013705379329621792, -0.0434904508292675, 0.001730396063067019, -0.004829535726457834, 0.07924166321754456, -0.04070029407739639, -0.04413606598973274, -0.05974854901432991, -0.024605747312307358, -0.0645090714097023, 0.04975520819425583, -0.02771110273897648, 0.04318665340542793, -0.007147501688450575, 0.05538639426231384, -0.004849149379879236, 0.0029466671403497458, -0.010538572445511818, -0.08563341200351715, 0.081639863550663, -0.060857318341732025, -0.046611834317445755, -0.024326113983988762, 0.021671846508979797, -0.016179624944925308, -0.003337707370519638, 0.05939548835158348, -0.04128606617450714, -0.07674407958984375, -0.07292202115058899, 0.0240772794932127, -0.05389765650033951, 0.03797171637415886, -0.02399475872516632, 0.016844255849719048, 0.05382734164595604, -0.07770473510026932, -0.06601886451244354, -0.041514188051223755, -0.06618429720401764, -0.07793647050857544, -0.05251377448439598, 0.034804314374923706, -0.023276761174201965, 0.0007365281344391406, 0.05444197729229927, 0.046772826462984085, -0.02057318203151226, 0.012025891803205013, 0.01487181056290865, 0.013869334943592548, -0.026398126035928726, 0.021863926202058792, -0.009852495044469833, 0.047106217592954636, 0.07984088361263275, 0.03999389335513115, 0.011065179482102394, -0.03992863744497299, 0.04079510271549225, 0.025790706276893616, 0.008685950189828873, 0.02779287099838257, 0.04385172575712204, -0.04277932271361351, 0.07673835754394531, -0.021691404283046722, 0.022119486704468727, -0.01066500786691904, -0.06706197559833527, -0.05776849389076233, 0.013126887381076813, -0.07057829946279526, -0.0032913032919168472, 0.03295627981424332, 0.04386640340089798, 0.03083283267915249, 0.05029971897602081, 0.03992484137415886, 0.05541689693927765, -0.009236439131200314, -0.031200051307678223, -0.00673228595405817, 0.010596859268844128, -0.012021317146718502, 0.07108088582754135, 0.029357079416513443, 0.0005943229771219194, 0.05980091542005539, 0.05378343537449837, -0.060533687472343445, 0.05232305824756622, 0.049413252621889114, -0.06799817085266113, -0.06145549938082695, -0.049923140555620193, -0.004467437509447336, 0.012020272202789783, 0.07885237783193588, -0.005360431503504515, -0.06863974034786224, 0.032793980091810226, -0.027088897302746773, -0.030258439481258392, 0.04452057182788849, 0.03226466849446297, 0.014056222513318062, -0.047953009605407715, -0.06128696724772453, 0.07707038521766663, -0.011983855627477169, 0.00987423025071621, -0.059393275529146194, -0.009502439759671688, 0.04373832792043686, -0.07663515210151672, -0.03551071882247925, -0.0792846754193306, -0.040732547640800476, 0.04964913800358772, 0.022374317049980164, 0.030285334214568138, 0.024618737399578094, -0.057423535734415054, 0.0374540314078331, -0.0679716095328331, 0.041351739317178726, -0.014735174365341663, -0.03132721036672592, -0.016562623903155327, -0.023142848163843155, -0.0003179792547598481, -0.04326152801513672, -0.018657725304365158, 0.07241569459438324, -0.05912894383072853, 0.0018715967889875174, -0.03247078135609627, -0.0036502541042864323, -0.010405274108052254, -0.020420199260115623, -0.0058896504342556, -0.06328868120908737, -0.012468191795051098, 0.04778258129954338, 0.04628447815775871, -0.04462042078375816, -0.02249697968363762, 0.08650689572095871, 0.004042086191475391, 0.006387397181242704, -0.08236270397901535, -0.042739491909742355, -0.044546108692884445, -0.04928244277834892, 0.08043301850557327, 0.024950409308075905, 0.040725886821746826, -0.035485439002513885, -0.0471055693924427, 0.02541366219520569, 0.026015281677246094, -0.024492928758263588, -0.00936790369451046, -0.01749337464570999, -0.028561461716890335, 0.07058209925889969, -0.021949201822280884, -0.08433210104703903, -0.05350934714078903, -0.010750233195722103, 0.04425276070833206, 0.03130744397640228, 0.02272653765976429]
sentenceSimilaritybyNN_300d(embeddings, es, query)

Semantic Similarity Search:

----------------------------------------
1.535937	Install Shield 2009 Premier : Install Windows Installer 3.1 or 2.0 (best fit for system) with any sysem restart <p>Basic MSI project, from release wizard I chose Install Windows Installer 3.1 or 2.0 (best fit for system), extract from setup.exe. When in installation, after installing Windows installer system restarts. Can this restart be stopped anyway?</p>

----------------------------------------
1.4687387	Install Shield 2009 - Shortcuts creation <p>Is it possible to create shortcuts in Install Shield 2009 based on user input?
In essnes, a shortcut will be created (on the desktop) only if the user (who runs the installation) wants to do so</p>

----------------------------------------
1.4648793	How to detect SQL Server Express in WiX installer <p>How do I detect if Sql Server Express is installed and running on a machine in a WiX installer?</p>

<p>I want to check before installing my application and if it

In [3]:
questions=pd.read_csv(r"/Users/amansawarn/Downloads/Questions.csv", encoding='ISO-8859-1')

### Questions from OwnerID : 875317.0


In [13]:

single_user_questions= [7948030, 8088820, 8199310, 8277170, 8425690, 8460990, 8556920, 8640530, 8650600, 8675080, 8737190, 8737270, 8778850, 8949900, 9122560, 9135110, 9153650, 9264090, 9284610, 9302050, 9753960, 9830940, 9897790, 9934900, 10220010, 10324780, 10377130, 10438220, 10689140, 10901590, 10902620, 10952470, 10990520, 11006100, 11386670, 11508240, 11514640, 11614800, 11857700, 11860540, 11939460, 11995470, 11996060, 12046810, 12201620, 12270710, 12410630, 12610160, 12644050, 13036850, 13130930, 13180180, 13517590, 13661440, 13769240, 13851460, 14007390, 14104040, 14342390, 14945720, 15077220, 15312500, 15313230, 15330310, 15503570, 15774160, 15912410, 15980680, 16019210, 16158440, 16406940, 16484560, 16509150, 16513530, 16547960, 16843420, 16991020, 17112660, 17227920, 17371790, 17391430, 17414390, 17431660, 17432960, 17540480, 17581330, 17599630, 17631690, 17758320, 17776800, 17952900, 17955040, 18242690, 18324370, 18346600, 18601400, 18857350, 18967210, 18991450, 19011120, 19525590, 19552970, 19827190, 20020930, 20059590, 20404720, 20479480, 20481590, 20508080, 20528610, 20752400, 20767520, 20792590, 20843810, 20909150, 20960690, 21247110, 21315210, 21840400, 21886350, 22056370, 22078830, 22082080, 22331090, 22487330, 22518930, 22773300, 22970470, 23089600, 23352970, 23375050, 23376460, 23412230, 23618530, 23745390, 23958330, 24024790, 24290170, 24336960, 24372620, 24642870, 24942190, 24966070, 25251890, 25557930, 25751970, 25754040, 25875930, 26197600, 26242460, 26332480, 26811690, 26817550, 26851470, 27283540, 27328490, 27390350, 27466820, 27804430, 27828650, 28120220, 28967930, 29102770, 29146860, 29268900, 29287830, 29525650, 29612810, 29615280, 29951130, 30035090, 30493670, 30518470, 30720010, 30926250, 31349890, 31392190, 31575300, 31731270, 31796950, 31889500, 32083660, 32156980, 32319800, 32681480, 32701200, 32719190, 32768280, 32770560, 32877570, 32908300, 32955160, 32997950, 33000100, 33087170, 33250970, 33335910, 33444970, 33619160, 33657500, 33682680, 33831580, 34028720, 34098970, 34187800, 34190950, 34405300, 34639830, 34727170, 34749680, 34774030, 34775570, 34908470, 34931090, 35182130, 35272440, 35282380, 35321220, 35495040, 35701810, 36092420, 36135150, 36367150, 36380980, 36585240, 36724560, 36754980, 36776260, 36991840, 36992470, 37263990, 37395210, 37467900, 37575920, 37580420, 37690970, 37708220, 37758170, 38106950, 38357250, 38793640, 38857740, 39089700, 39414590, 39583430, 39645440, 39646320, 39779510, 39925460, 40117250]
## Refer Data Processing Notebook to see how we arrived at these ids


In [23]:
pd.set_option('display.max_colwidth', None)


In [16]:
test=questions[questions['Id'].isin(single_user_questions)]

In [24]:
test['total_text']=test['Title']+ " "+ test['Body']

/var/folders/zn/4zzj_7js7351yjh2d0dss37h0000gn/T/ipykernel_31732/1015250534.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['total_text']=test['Title']+ " "+ test['Body']


In [25]:
test

Id  OwnerUserId          CreationDate            ClosedDate  \
209813    7948030     875317.0  2011-10-30T21:39:13Z  2013-03-03T14:42:19Z   
214102    8088820     875317.0  2011-11-11T01:57:17Z                   NaN   
217361    8199310     875317.0  2011-11-20T03:47:43Z                   NaN   
219742    8277170     875317.0  2011-11-26T08:12:10Z                   NaN   
224220    8425690     875317.0  2011-12-08T03:16:35Z                   NaN   
...           ...          ...                   ...                   ...   
1243282  39645440     875317.0  2016-09-22T17:35:09Z                   NaN   
1243321  39646320     875317.0  2016-09-22T18:25:17Z                   NaN   
1248697  39779510     875317.0  2016-09-29T20:34:01Z                   NaN   
1254885  39925460     875317.0  2016-10-07T20:39:21Z                   NaN   
1263146  40117250     875317.0  2016-10-18T20:11:38Z                   NaN   

         Score  \
209813       0   
214102      17   
217361       0   
219742       1   
224220       0   
...        ...   
1243282      0   
1243321      0   
1248697      0   
1254885     12   
1263146      1   

                                                                                                                                                    Title  \
209813                                                                              Shouldn't the Manifest file be named "AndroidManifest" in the editor?   
214102                                                                     Can I set Eclipse to only emit errors, warnings, etc. for the current project?   
217361                                                                                                     The Case of the Incredible Disappearing Button   
219742                                                                                                            How can I run my app without debugging?   
224220                                                                Why is my value saved to SharedPreferences in onPause() not restored in onResume()?   
...                                                                                                                                                   ...   
1243282                                                                            How can I cause a legend to appear to the right of the pie (Chart.JS)?   
1243321                                      How can I show a subset of data on pie pieces in Chart.JS while still displaying the superset when hovering?   
1248697                                                             How can I calculate a value and place it inside the last bar in a Chart.JS bar chart?   
1254885                                                                                            How can I filter columns (rather than rows) in EPPlus?   
1263146  Why is some of the colorization of cells not occurring, and is there a connection between that and the "Cannot perform runtime binding" err msg?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [30]:
query=" ".join(test['total_text'].to_list())

In [35]:
for i in test['total_text'].to_list()[:3]:
    print(i)
    print("-"*40)

Shouldn't the Manifest file be named "AndroidManifest" in the editor? <p>I'm getting a nonsensical error message:</p>

<blockquote>
  <p>The element type "manifest" must be terminaed by the mathing end-tag <code>&lt;/manifest&gt;</code>.</p>
</blockquote>

<p>This is odd because that is not a problem in the manifest file. What IS odd, though, is that although the file is named <code>AndroidManifest.xml</code> in the Package Explorer, when I have it open in the editor its name there is <code>&lt;ProjectName&gt; Manifest</code>.</p>

<p>It should read "AndroidManifest" right?</p>

<p>If so, how can I get it to know that it is that, and not <code>&lt;Project Name&gt; Manifest</code>?</p>

----------------------------------------
Can I set Eclipse to only emit errors, warnings, etc. for the current project? <p>I want to be able to ignore warnings in other projects in my workspace when I'm concentrating on my current project. I looked through Window | Preferences and saw no such option. Ecl

In [31]:
# query="how to install pip?"
embeddings=list(get_glove_vectors_from_sentence(query, glove_model))
# embeddings=[-0.03462885320186615, 0.00767511036247015, 0.004583781119436026, 0.028401656076312065, 0.01258331723511219, 0.037116874009370804, 0.013477616012096405, 0.022381523624062538, 0.03979786857962608, 0.024851974099874496, 0.036737024784088135, 0.03597935289144516, -0.0381510891020298, 0.08130659908056259, -0.05906254053115845, 0.08973553776741028, -0.03244425728917122, 0.051288917660713196, 0.02930707484483719, 0.022335924208164215, 0.020585650578141212, 0.04718436673283577, -0.05237151309847832, -0.043897032737731934, 0.015714094042778015, 0.02331877127289772, 0.03893255814909935, -0.05679837986826897, 0.03729335218667984, -0.049225278198719025, 0.06690701842308044, 0.07899188995361328, 0.016306092962622643, -0.02428351156413555, 0.019640468060970306, 0.03080165386199951, 0.07699830085039139, -0.014750823378562927, -0.051929473876953125, 0.0391777828335762, 0.034724123775959015, -0.03729329630732536, 0.0199520792812109, -0.019983848556876183, 0.06893950700759888, -0.01282433234155178, -0.018197476863861084, -0.0725010335445404, 0.08238957822322845, 0.038134943693876266, 0.04725194349884987, 0.08184657990932465, -0.038238804787397385, -0.040010299533605576, -0.05526154115796089, -0.04314691573381424, -0.017179878428578377, -0.04188328608870506, 0.06424654275178909, 0.08285296708345413, 0.013338451273739338, 0.02647676318883896, 0.0059562912210822105, -0.027724098414182663, 0.03596652299165726, -0.0713033601641655, -0.040550850331783295, -0.054018519818782806, -0.05968989431858063, 0.05485939979553223, 0.0054743546061217785, 0.04592319577932358, 0.04088946059346199, 0.009760835207998753, -0.024534588679671288, 0.03911426663398743, 0.034169845283031464, -0.009666499681770802, 0.04447918012738228, 0.05167559161782265, -0.025161610916256905, 0.05686209723353386, -0.024312159046530724, 0.0038878333289176226, -0.06046237796545029, 0.00990272406488657, 0.0021283847745507956, 0.035826608538627625, 0.07240117341279984, -0.03567688912153244, -0.07299879193305969, 0.0098195755854249, -0.07013959437608719, 0.05116822198033333, -0.07229223847389221, -0.008418512530624866, 0.034077126532793045, 0.06645393371582031, 0.027137843891978264, -0.08534674346446991, 0.03342875465750694, 0.0010534467874094844, -0.07039080560207367, -0.061267413198947906, 0.00017997882969211787, 0.0448671318590641, 0.002362641738727689, 0.030875999480485916, 0.05316605791449547, -0.04655775800347328, 0.055160440504550934, -0.014520411379635334, -0.03851637616753578, -0.023013180121779442, -0.043779704719781876, 0.02196790650486946, 0.019681503996253014, 0.025772398337721825, -0.07112425565719604, -0.041777562350034714, -0.08362290263175964, 0.011219008825719357, 0.041559576988220215, 0.042125992476940155, -0.028749441727995872, -0.026517637073993683, -0.015619627200067043, 0.0024774977937340736, 0.04005668684840202, 0.06414245814085007, -0.0193956159055233, 0.05626853182911873, -0.006098745856434107, -0.027311531826853752, -0.0020660171285271645, -0.055144451558589935, -0.0790662094950676, 0.013367146253585815, 0.013629199005663395, 0.032548774033784866, -0.052126847207546234, -0.07231476902961731, 0.045602746307849884, -0.022876564413309097, -0.01824858784675598, -0.07567820698022842, -0.05951853096485138, -0.0033317203633487225, -0.019438941031694412, -0.07292303442955017, 0.039319004863500595, -0.030262252315878868, 0.023188702762126923, 0.049401767551898956, -0.03795567527413368, 0.00596885709092021, 0.07696520537137985, -0.06870242953300476, -0.023071812465786934, 0.032319311052560806, 0.027019716799259186, 0.01199339423328638, -0.054047685116529465, -0.034860048443078995, 0.004286427516490221, 0.031746670603752136, -0.03668864071369171, 0.05324863642454147, -0.014945111237466335, 0.012569617480039597, -0.014295347034931183, -0.05456634238362312, -0.02866458334028721, -0.031186982989311218, 0.011176125146448612, -0.04290825501084328, -0.02617817372083664, -0.014771874062716961, -0.02310779318213463, 0.03021686151623726, 0.02180018089711666, -0.06680597364902496, -0.013131853193044662, 0.07586025446653366, 0.0587407648563385, 0.021105261519551277, -0.043579600751399994, -0.05390194058418274, -0.006434908602386713, 0.011121878400444984, 0.043450430035591125, -0.046666622161865234, 0.03760552406311035, 0.057415563613176346, -0.02668597176671028, 0.0025108535774052143, 0.07808926701545715, 0.09018489718437195, 0.03663500025868416, -0.049429137259721756, -0.05500154197216034, 0.0045248595997691154, -0.0663011223077774, 0.0767505019903183, -0.02247614786028862, -0.04814096540212631, 0.03155306726694107, 0.048325467854738235, -0.0317516028881073, 0.07029253989458084, -0.014762896113097668, 0.039758119732141495, -0.027383100241422653, 0.01077291090041399, -0.018674813210964203, 0.06772593408823013, -0.019307976588606834, 0.06167430803179741, -0.04702961817383766, 0.015139950439333916, 0.0035179078113287687, 0.03386974334716797, -0.027051519602537155, 0.03885263204574585, 0.03370879217982292, 0.030938170850276947, 0.03143477067351341, 0.033542290329933167, -0.06624799966812134, -0.04648910462856293, -0.04922991245985031, 0.03850280120968819, 0.03083956614136696, 0.0036689413245767355, -0.06498728692531586, 0.011056188493967056, -0.014391270466148853, -0.007774649187922478, -0.00781356543302536, 0.04289641231298447, 0.04494582489132881, -0.04518967121839523, 0.060684725642204285, -0.023170430213212967, 0.06990323215723038, 0.0877854973077774, 0.0027513860259205103, -0.05976095050573349, 0.09043174982070923, 0.013956253416836262, -0.09000561386346817, -0.015043241903185844, -0.03513569012284279, 0.01768846996128559, -0.04067947715520859, -0.014423196204006672, -0.030041474848985672, -0.06351054459810257, 0.02664155326783657, 0.03906116634607315, 0.0147284846752882, 0.01246387418359518, -0.09055300801992416, 0.07144160568714142, -0.05196592956781387, -0.020227991044521332, 0.0341203548014164, -0.08681684732437134, 0.02282768115401268, -0.004371015354990959, 0.011171799153089523, -0.017510192468762398, 0.08090528100728989, -0.041865479201078415, -0.01456389483064413, -0.020004156976938248, -0.009722404181957245, 0.03202565386891365, 0.0694165900349617, -0.02828393317759037, -0.032582838088274, -0.011106813326478004, -0.030484862625598907, 0.009067831560969353, -0.020938389003276825, -0.0230411347001791, -0.07012031972408295, 0.0541088804602623, -0.042449284344911575, 0.018649384379386902, 0.03883485123515129, -0.048480186611413956, 0.0344393365085125, -0.06893027573823929, 0.08569465577602386, -0.06986629217863083, 0.008478878997266293, 0.06473923474550247, -0.07792525738477707, 0.001722044195048511, 0.01306208036839962, 0.04006865620613098, -0.06954129040241241, -0.021065140143036842, -0.005601787474006414, 0.0076944660395383835, 0.032397426664829254, 0.057154640555381775, -0.05126655101776123, -0.012916426174342632, 0.03863914683461189, 0.04436832293868065, 0.009613048285245895, -0.03996099904179573, -0.0003503428597468883, 0.031744007021188736, -0.00216520088724792, 0.017528623342514038, -0.06498982012271881, 0.07106641680002213, -0.04223049059510231, 0.03721412643790245, -0.0073788901790976524, 0.06726673245429993, -0.03244607150554657, 0.05853502079844475, 0.005853424314409494, 0.018217280507087708, -0.0048925606533885, 0.07971342653036118, -0.04592495411634445, 0.05102983117103577, 0.07912196218967438, 0.07316495478153229, -0.017797192558646202, 0.048888444900512695, -0.06103556603193283, -0.03492056950926781, 0.013705379329621792, -0.0434904508292675, 0.001730396063067019, -0.004829535726457834, 0.07924166321754456, -0.04070029407739639, -0.04413606598973274, -0.05974854901432991, -0.024605747312307358, -0.0645090714097023, 0.04975520819425583, -0.02771110273897648, 0.04318665340542793, -0.007147501688450575, 0.05538639426231384, -0.004849149379879236, 0.0029466671403497458, -0.010538572445511818, -0.08563341200351715, 0.081639863550663, -0.060857318341732025, -0.046611834317445755, -0.024326113983988762, 0.021671846508979797, -0.016179624944925308, -0.003337707370519638, 0.05939548835158348, -0.04128606617450714, -0.07674407958984375, -0.07292202115058899, 0.0240772794932127, -0.05389765650033951, 0.03797171637415886, -0.02399475872516632, 0.016844255849719048, 0.05382734164595604, -0.07770473510026932, -0.06601886451244354, -0.041514188051223755, -0.06618429720401764, -0.07793647050857544, -0.05251377448439598, 0.034804314374923706, -0.023276761174201965, 0.0007365281344391406, 0.05444197729229927, 0.046772826462984085, -0.02057318203151226, 0.012025891803205013, 0.01487181056290865, 0.013869334943592548, -0.026398126035928726, 0.021863926202058792, -0.009852495044469833, 0.047106217592954636, 0.07984088361263275, 0.03999389335513115, 0.011065179482102394, -0.03992863744497299, 0.04079510271549225, 0.025790706276893616, 0.008685950189828873, 0.02779287099838257, 0.04385172575712204, -0.04277932271361351, 0.07673835754394531, -0.021691404283046722, 0.022119486704468727, -0.01066500786691904, -0.06706197559833527, -0.05776849389076233, 0.013126887381076813, -0.07057829946279526, -0.0032913032919168472, 0.03295627981424332, 0.04386640340089798, 0.03083283267915249, 0.05029971897602081, 0.03992484137415886, 0.05541689693927765, -0.009236439131200314, -0.031200051307678223, -0.00673228595405817, 0.010596859268844128, -0.012021317146718502, 0.07108088582754135, 0.029357079416513443, 0.0005943229771219194, 0.05980091542005539, 0.05378343537449837, -0.060533687472343445, 0.05232305824756622, 0.049413252621889114, -0.06799817085266113, -0.06145549938082695, -0.049923140555620193, -0.004467437509447336, 0.012020272202789783, 0.07885237783193588, -0.005360431503504515, -0.06863974034786224, 0.032793980091810226, -0.027088897302746773, -0.030258439481258392, 0.04452057182788849, 0.03226466849446297, 0.014056222513318062, -0.047953009605407715, -0.06128696724772453, 0.07707038521766663, -0.011983855627477169, 0.00987423025071621, -0.059393275529146194, -0.009502439759671688, 0.04373832792043686, -0.07663515210151672, -0.03551071882247925, -0.0792846754193306, -0.040732547640800476, 0.04964913800358772, 0.022374317049980164, 0.030285334214568138, 0.024618737399578094, -0.057423535734415054, 0.0374540314078331, -0.0679716095328331, 0.041351739317178726, -0.014735174365341663, -0.03132721036672592, -0.016562623903155327, -0.023142848163843155, -0.0003179792547598481, -0.04326152801513672, -0.018657725304365158, 0.07241569459438324, -0.05912894383072853, 0.0018715967889875174, -0.03247078135609627, -0.0036502541042864323, -0.010405274108052254, -0.020420199260115623, -0.0058896504342556, -0.06328868120908737, -0.012468191795051098, 0.04778258129954338, 0.04628447815775871, -0.04462042078375816, -0.02249697968363762, 0.08650689572095871, 0.004042086191475391, 0.006387397181242704, -0.08236270397901535, -0.042739491909742355, -0.044546108692884445, -0.04928244277834892, 0.08043301850557327, 0.024950409308075905, 0.040725886821746826, -0.035485439002513885, -0.0471055693924427, 0.02541366219520569, 0.026015281677246094, -0.024492928758263588, -0.00936790369451046, -0.01749337464570999, -0.028561461716890335, 0.07058209925889969, -0.021949201822280884, -0.08433210104703903, -0.05350934714078903, -0.010750233195722103, 0.04425276070833206, 0.03130744397640228, 0.02272653765976429]
sentenceSimilaritybyNN_300d(embeddings, es, query)

Semantic Similarity Search:

----------------------------------------
1.97957	How to handle checkboxes in ASP.NET MVC forms? <h2>Caution:  This question is over five years old!</h2>

<p>Your best option is to search for newer questions, or to search the answers below looking for your specific version of MVC, as many answers here are obsolete now.  </p>

<p>If you do find an answer that works for your version, please make sure the answer contains the version of MVC you are using.<br>
<sub><sub>(The original question starts below)</sub></sub></p>

<hr>

<p>This seems a bit bizarre to me, but as far as I can tell, this is how you do it.</p>

<p>I have a collection of objects, and I want users to select one or more of them.  This says to me "form with checkboxes."  My objects don't have any concept of "selected" (they're rudimentary POCO's formed by deserializing a wcf call).  So, I do the following:</p>

<pre><code>public class SampleObject{
  public Guid Id {get;set;}
  public string Nam